In [1]:
###Importing packages we need###

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from matplotlib.lines import Line2D
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import matplotlib.patches as patches
import cdsapi
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import metpy as mp
import os
import warnings
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [21]:
###Import the data###

#TCWV_LP = xr.open_dataset('tcwv_comp_LP.nc') 
#TCWV_HP = xr.open_dataset('tcwv_comp_HP.nc')

TCWV_LP = xr.open_dataset('tcwv_comp_LP_8hr.nc')  #can change
TCWV_HP = xr.open_dataset('tcwv_comp_HP_8hr.nc')

TCWV_HP

<xarray.Dataset>
Dimensions:        (case: 145, latitude: 33, longitude: 45)
Coordinates:
  * case           (case) int64 0 1 2 3 4 5 6 7 ... 138 139 140 141 142 143 144
  * latitude       (latitude) float64 34.0 34.31 34.62 ... 43.38 43.69 44.0
  * longitude      (longitude) float64 -112.0 -111.7 -111.4 ... -99.3 -99.0
Data variables:
    composite_var  (case, latitude, longitude) float32 ...
Attributes:
    title:        Cold-front composites
    source:       your script
    Conventions:  CF-1.8

In [22]:
###Figure out significance for each group for all of eastern CO###

#1) Define Eastern Colorado bounds#
lat_min, lat_max = 37.0, 41.0
lon_min, lon_max = -105.25, -102.00

#2) Subset each dataset to Eastern Colorado#
TCWV_HP_eCO = TCWV_HP.sel(latitude=slice(lat_min, lat_max),
                          longitude=slice(lon_min, lon_max))

TCWV_LP_eCO = TCWV_LP.sel(latitude=slice(lat_min, lat_max),
                          longitude=slice(lon_min, lon_max))

#3) Average over lat/lon to get one value per case#
hp_avg = TCWV_HP_eCO['composite_var'].mean(dim=['latitude', 'longitude'])
lp_avg = TCWV_LP_eCO['composite_var'].mean(dim=['latitude', 'longitude'])

#4) Compute fractions#
frac_HP_pos = (hp_avg > 0).sum().item() / hp_avg.size
frac_LP_neg = (lp_avg < 0).sum().item() / lp_avg.size

print(f"Fraction of HP cases with positive TCWV anomaly: {frac_HP_pos:.2f}")
print(f"Fraction of LP cases with negative TCWV anomaly: {frac_LP_neg:.2f}")

Fraction of HP cases with positive TCWV anomaly: 0.80
Fraction of LP cases with negative TCWV anomaly: 0.66


In [23]:
###Figure out significance for each group for single rep point###

#Target point in Eastern Colorado#
target_lat = 38
target_lon = -103

#Select the nearest gridpoint in each dataset#
TCWV_HP_point = TCWV_HP.sel(latitude=target_lat, longitude=target_lon, method='nearest')
TCWV_LP_point = TCWV_LP.sel(latitude=target_lat, longitude=target_lon, method='nearest')

#Extract the time series (one value per case)#
hp_vals = TCWV_HP_point['composite_var']
lp_vals = TCWV_LP_point['composite_var']

#Fractions#
frac_HP_pos = (hp_vals > 0).sum().item() / hp_vals.size
frac_LP_neg = (lp_vals < 0).sum().item() / lp_vals.size

print(f"Gridpoint location HP: lat={TCWV_HP_point.latitude.item():.2f}, lon={TCWV_HP_point.longitude.item():.2f}")
print(f"Gridpoint location LP: lat={TCWV_LP_point.latitude.item():.2f}, lon={TCWV_LP_point.longitude.item():.2f}")
print(f"Fraction of HP cases with positive TCWV anomaly: {frac_HP_pos:.2f}")
print(f"Fraction of LP cases with negative TCWV anomaly: {frac_LP_neg:.2f}")

Gridpoint location HP: lat=38.06, lon=-103.14
Gridpoint location LP: lat=38.06, lon=-103.14
Fraction of HP cases with positive TCWV anomaly: 0.84
Fraction of LP cases with negative TCWV anomaly: 0.56


In [12]:
TCWV_HP

<xarray.Dataset>
Dimensions:        (case: 145, latitude: 33, longitude: 45)
Coordinates:
  * case           (case) int64 0 1 2 3 4 5 6 7 ... 138 139 140 141 142 143 144
  * latitude       (latitude) float64 34.0 34.31 34.62 ... 43.38 43.69 44.0
  * longitude      (longitude) float64 -112.0 -111.7 -111.4 ... -99.3 -99.0
Data variables:
    composite_var  (case, latitude, longitude) float32 ...
Attributes:
    title:        Cold-front composites
    source:       your script
    Conventions:  CF-1.8